In [40]:
_author_ = "Rahul_Chowdhury"
import numpy as np
import pandas as pd
import sys
from collections import defaultdict
print(sys.version)

3.5.2 |Anaconda 4.2.0 (64-bit)| (default, Jul  5 2016, 11:41:13) [MSC v.1900 64 bit (AMD64)]


In [19]:
data_url =("https://archive.ics.uci.edu/ml/machine-learning-databases/tae/tae.data")
Data = pd.read_csv(data_url, header=None, prefix="V")
k = len(list(Data['V5'].unique())) #counting the number of classes in the last column
Data = Data.drop(Data.columns[5], axis=1)
Data.head(10)
#print("Number of classes:",k)



,V0,V1,V2,V3,V4
0,1,23,3,1,19
1,2,15,3,1,17
2,1,23,3,2,49
3,1,5,2,2,33
4,2,7,11,2,55
5,2,23,3,1,20
6,2,9,5,2,19
7,2,10,3,2,27
8,1,22,3,1,58
9,2,15,3,1,20


In [67]:
sum_unique_values = 0
for col in Data:
    if (col!= 'V4'): #V4 is numerical data
        unique_values_in_col = len(list(Data[col].unique()))
        #print(unique_values_in_col)
        sum_unique_values += unique_values_in_col
avg_sum_unique_values = round(sum_unique_values/(len(Data.columns)-1))
print(avg_sum_unique_values)
num_list = list(Data['V4'].unique())
num_list.sort()
print(len(num_list))
num_list = np.array_split(num_list,int(len(num_list)/avg_sum_unique_values))
equiv_class_numerical ={}
equiv_class_numerical_index = 1
for item in num_list:
    equiv_class_numerical[equiv_class_numerical_index] = list(item)
    equiv_class_numerical_index += 1
    
print(equiv_class_numerical)
new_column = []
for item in Data['V4']:
    for key in equiv_class_numerical:
        if(item in equiv_class_numerical[key]):
            new_column.append(key)
Data['V4_class'] = new_column
Data

12
46
{1: [3, 5, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21], 2: [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36], 3: [37, 38, 39, 42, 43, 44, 45, 46, 48, 49, 51, 54, 55, 58, 66]}


,V0,V1,V2,V3,V4,V4_class
0,1,23,3,1,19,1
1,2,15,3,1,17,1
2,1,23,3,2,49,3
3,1,5,2,2,33,2
4,2,7,11,2,55,3
5,2,23,3,1,20,1
6,2,9,5,2,19,1
7,2,10,3,2,27,2
8,1,22,3,1,58,3
9,2,15,3,1,20,1


In [52]:
def MMeR(U,k):
    C =  {}#the clusters made through MMeR alogrithm
    C_distance = {}
    #Rough_Data = U
    Rough_Data = U.copy()
    #print(Rough_Data)
    index = 0
   
    while(k>len(C.keys())):
        #print(Rough_Data) 
        split_attr,min_roughness_attr_value = mmer_roughness(Rough_Data)
        print(split_attr)
        #min_roughness_attr_value = min(roughness_list_dict[split_attr]) #find the min. value of min_roughness_attr for which alpha is minimum
        #print(min_roughness_attr_value)
        cluster1 = Rough_Data[Rough_Data[split_attr] == min_roughness_attr_value].index.tolist()
        cluster1_distance = cluster_distance(cluster1, Data)
        cluster2 = Rough_Data.loc[Rough_Data[split_attr] != min_roughness_attr_value].index.tolist()
        cluster2_distance = cluster_distance(cluster2, Data)
        C[index] = cluster1
        C_distance[index] = cluster1_distance
        #print(cluster1_distance)
        #print(cluster2_distance)
        index += 1
        C[index] = cluster2
        C_distance[index] = cluster2_distance
        index += 1
        max_index = max(C_distance, key= C_distance.get) #find the index with the max cluster distance
        
              
        Rough_Data = U.ix[C[max_index]]  #send data of cluster with max distance
        for key in C:
            print("Cluster",C[key])
            print("Cluster distance", C_distance[key])
      
        
        if(len(C.keys())!= (k-1)):
               
            del C[max_index]
            del C_distance[max_index]
        
        print("C length:",len(C.keys()))
 


def roughness(col_name1, col_name2, val, Data):
     
     equiv_class_dict = {} #dictionary storing equiv classes for all attribute
     for col in Data:
        equiv_class_dict[col] =list(Data.groupby([col]))
        
     arr = equiv_class_dict[col_name2] #group equiv classes according to column2
     target_set = Data[Data[col_name1] == val].index.tolist() #calculating target set for each unique value in column1
     lower_approx = []
     upper_approx = []
     for name,group in arr:
       selected_list = list(group.index)
       if(set(selected_list).issubset(target_set)): #if a subset, then append to lower_approx
            lower_approx.append(selected_list)
       if(set(selected_list) & set(target_set)): #if intersection is not null, then append to upper_approx
            upper_approx.append(selected_list)
     lower_approx_count = sum(map(len, lower_approx)) #count the no. of elements in lower_approx
     upper_approx_count = sum(map(len, upper_approx))
     if(lower_approx_count == 0):
        roughness = 1
     else:   
         roughness = 1 - (lower_approx_count/upper_approx_count) # return roughness for a(i) = alpha
     return roughness



def cluster_distance(cluster, Data):
        distance = 0
        for i in range(0,len(cluster)-1):
              a = np.array(Data.ix[cluster[i]])
              for k in range(i+1,len(cluster)):
                   b = np.array(Data.ix[cluster[k]])
                   distance += sum(a != b) 
        print(len(cluster))  
        final_distance = (2 * distance)/(len(cluster) * (len(cluster)-1)) 
                  
        return final_distance

    
def mmer_roughness(Data):
    #check if all columns have more than 1 unique values, if not drop that column/columns
    #print("Dropped columns:")
    #print(Data)
    for col1 in Data:
         values_in_col_name1 =  list(Data[col1].unique()) #extracting each unique value in column1
         #print(col1, len(values_in_col_name1))
         if(len(values_in_col_name1) == 1):
            Data = Data.drop(col1, axis = 1)
            #print(col1, ",")
   
    
    min_mean_roughness = {} #array to store the minimum of average of roughness of each attr
    roughness_list_dict = defaultdict(dict) #needed to use 2D dictionary 
   
    
    
    for col1 in Data:
        values_in_col_name1 =  list(Data[col1].unique()) #extracting each unique value in column1
        mean_roughness = [] #list for mean roughness for a(i) = alpha
        for val in values_in_col_name1:
            roughness_list = [] #array of a(i) = alpha w.r.t. other columns 
            for col2 in Data:
                if(col1!= col2): #not to be compared with itself
                  roughness_list.append(roughness(col1 , col2, val, Data))
            mean_roughness.append(sum(roughness_list)/float(len(roughness_list)))
            roughness_list_dict[col1][val] = sum(roughness_list)/float(len(roughness_list))
        min_mean_roughness[col1]=(min(mean_roughness))
    #for key in min_mean_roughness:
        #print(key, min_mean_roughness[key]) 
    #print(min_mean_roughness)    
    min_roughness_attr = min(min_mean_roughness, key=min_mean_roughness.get) #find the attr with minimum roughness
    #print(min_roughness_attr)
    #print(min_roughness_attr,  min(roughness_list_dict[min_roughness_attr]))

    return min_roughness_attr,  min(roughness_list_dict[min_roughness_attr])
     

    


MMeR(Data,k) 


V3
23
128
Cluster [0, 1, 5, 8, 9, 18, 23, 33, 39, 40, 44, 48, 57, 62, 72, 78, 79, 82, 85, 86, 87, 101, 102]
Cluster distance 2.34387351779
Cluster [2, 3, 4, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 41, 42, 43, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 73, 74, 75, 76, 77, 80, 81, 83, 84, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150]
Cluster distance 3.11318897638
C length: 2
V0
20
108
Cluster [0, 1, 5, 8, 9, 18, 23, 33, 39, 40, 44, 48, 57, 62, 72, 78, 79, 82, 85, 86, 87, 101, 102]
Cluster distance 2.34387351779
Cluster [2, 3, 4, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35